In [1]:
from google.colab import userdata
import os

# 從 Colab Secrets 讀取 OPENAI_API_KEY 並設為環境變數
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [2]:
!pip install langgraph langchain-openai langchain-core ta yfinance python-dotenv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.9 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=ac6293e8cf9a5f78c5a289ccc70e468d71a6cf8082e34e547190e888c3e94c1e
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.37
    Uninstalling langchain-core-0.3.37:
      Successfully uninstalled langchain-core-0.3.37


In [3]:
FUNDAMENTAL_ANALYST_PROMPT = """
You are a fundamental analyst specializing in evaluating company (whose symbol is {company}) performance based on stock prices, technical indicators, financial metrics, recent news, industry trends, competitor positioning, and financial ratios. Your task is to provide a comprehensive summary.

You have access to the following tools:
1. **get_stock_prices**: Retrieves stock price data and technical indicators.
2. **get_financial_metrics**: Retrieves key financial metrics and financial ratios.
3. **get_financial_news**: Retrieves the latest financial news related to the stock.
4. **get_industry_data** *(if available)*: Retrieves industry trends and competitive positioning information.

---

### Your Task:
1. Use the provided stock symbol to query the tools.
2. Analyze the following areas in sequence:
   - **Stock price movements and technical indicators**: Examine recent price trends, volatility, and signals from RSI, MACD, VWAP, and other indicators.
   - **Financial health and key financial ratios**: Assess profitability, liquidity, solvency, and operational efficiency using metrics such as:
     - Profitability Ratios: Gross Profit Margin, Net Profit Margin, Operating Profit Margin
     - Liquidity Ratios: Current Ratio, Quick Ratio
     - Solvency Ratios: Debt-to-Equity Ratio, Interest Coverage Ratio
     - Efficiency Ratios: Inventory Turnover, Accounts Receivable Turnover
     - Market Ratios: Price-to-Earnings Ratio (P/E), Price-to-Book Ratio (P/B)
   - **Recent news and market sentiment**: Identify significant events or trends impacting the company's market perception.
   - **Industry analysis**: Evaluate the industry’s growth trends, technological advancements, and regulatory environment. Identify how the industry is evolving and how it affects the target company.
   - **Competitor analysis**: Compare the target company with key competitors in terms of market share, financial health, and growth potential.

3. Provide a concise and structured summary covering all sections, ensuring each area has actionable insights.

---

### Output Format : 以下請用繁體中文輸出
{
  "stock": "<Stock Symbol>",
  "price_analysis": "<股票價格趨勢與技術指標分析>",
  "technical_analysis": "<技術指標分析與見解>",
  "financial_analysis": {
      "profitability_ratios": "<獲利能力比率分析>",
      "liquidity_ratios": "<流動性比率分析>",
      "solvency_ratios": "<償債能力比率分析>",
      "efficiency_ratios": "<營運效率比率分析>",
      "market_ratios": "<市場表現比率分析>",
      "summary": "<財務整體健康狀況與分析結論>"
  },
  "news_analysis": "<近期新聞摘要與其對股價的潛在影響>",
  "industry_analysis": "<產業趨勢、成長動力與潛在風險>",
  "competitor_analysis": "<主要競爭對手比較與市場地位分析>",
  "final_summary": "<整體綜合結論與投資建議>",
  "Asked Question Answer": "<根據上述分析的具體回答>"
}

---

### Guidelines:
- Use the provided tools for data. If any data is unavailable, clearly state so in the respective section.
- Ensure the analysis is objective, data-driven, and free of speculative language.
- Keep responses concise but informative. Highlight actionable insights and risks.
- Output should be structured, easy to read, and in Traditional Chinese.
"""

from typing import Union, Dict, Set, List, TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
import yfinance as yf
import datetime as dt
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.volume import volume_weighted_average_price
import traceback
import pandas as pd
import dotenv
dotenv.load_dotenv()

@tool
def get_stock_prices(ticker: str) -> Union[Dict, str]:
    """Fetches historical stock price data and technical indicator for a given ticker."""
    try:
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(weeks=13),
            end=dt.datetime.now(),
            interval='1d'
        )
        df= data.copy()
        if len(df.columns[0]) > 1:
            df.columns = [i[0] for i in df.columns]
        data.reset_index(inplace=True)
        data.Date = data.Date.astype(str)

        indicators = {}

        # Momentum Indicators
        rsi_series = RSIIndicator(df['Close'], window=14).rsi().iloc[-12:]
        indicators["RSI"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in rsi_series.dropna().to_dict().items()}
        sto_series = StochasticOscillator(
            df['High'], df['Low'], df['Close'], window=14).stoch().iloc[-12:]
        # print(sto_series)
        indicators["Stochastic_Oscillator"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in sto_series.dropna().to_dict().items()}

        macd = MACD(df['Close'])
        macd_series = macd.macd().iloc[-12:]
        # print(macd_series)
        indicators["MACD"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in macd_series.to_dict().items()}
        macd_signal_series = macd.macd_signal().iloc[-12:]
        # print(macd_signal_series)
        indicators["MACD_Signal"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in macd_signal_series.to_dict().items()}

        vwap_series = volume_weighted_average_price(
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            volume=df['Volume'],
        ).iloc[-12:]
        indicators["vwap"] = {date.strftime('%Y-%m-%d'): int(value) for date, value in vwap_series.to_dict().items()}

        return {'stock_price': data.to_dict(orient='records'), 'indicators': indicators}
    except Exception as e:
        return f"Error fetching price data: {str(e)}"

@tool
def get_financial_news(ticker: str) -> Union[Dict, str]:
    """Fetches the latest financial news related to a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        news = stock.news  # 從 Yahoo Finance 獲取新聞
        if not news:
            return {"news": "No recent news found."}

        # 只取最新5則新聞
        latest_news = [
            {
                "title": item.get('title'),
                "publisher": item.get('publisher'),
                "link": item.get('link'),
                "published_date": item.get('providerPublishTime')
            }
            for item in news[:5]
        ]
        return {"news": latest_news}
    except Exception as e:
        return f"Error fetching news: {str(e)}"

def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'pe_ratio': info.get('forwardPE'),
            'price_to_book': info.get('priceToBook'),
            'debt_to_equity': info.get('debtToEquity'),
            'profit_margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching ratios: {str(e)}"


class State(TypedDict):
    messages: Annotated[list, add_messages]
    stock: str

graph_builder = StateGraph(State)

tools = [get_stock_prices, get_financial_metrics]
llm = ChatOpenAI(model='gpt-4o')
llm_with_tool = llm.bind_tools(tools)


# def fundamental_analyst(state: State):
#     messages = [
#         SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(company=state['stock'])),
#     ]  + state['messages']
#     return {
#         'messages': llm_with_tool.invoke(messages)
#     }
def fundamental_analyst(state: State):
    # 直接將 {company} 替換為 ticker
    prompt = f"{FUNDAMENTAL_ANALYST_PROMPT}".replace("{company}", state['stock'])

    messages = [
        SystemMessage(content=prompt),
    ] + state['messages']

    return {'messages': llm_with_tool.invoke(messages)}

graph_builder.add_node('fundamental_analyst', fundamental_analyst)
graph_builder.add_edge(START, 'fundamental_analyst')
graph_builder.add_node(ToolNode(tools))
graph_builder.add_conditional_edges('fundamental_analyst', tools_condition)
graph_builder.add_edge('tools', 'fundamental_analyst')

# HumanMessage(content=)
# graph_builder.add_edge('fundamental_analyst', END)
graph = graph_builder.compile()
events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
 'stock': '6669.TW'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()



================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================
Tool Calls:
  get_stock_prices (call_qkj9DLac1rlGHst25GxQFSU2)
 Call ID: call_qkj9DLac1rlGHst25GxQFSU2
  Args:
    ticker: 6669.TW
  get_financial_metrics (call_ViybSBNunS5Z2ENpHdAU7kkd)
 Call ID: call_ViybSBNunS5Z2ENpHdAU7kkd
  Args:
    ticker: 6669.TW
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 14.47773, "price_to_book": 4.710201, "debt_to_equity": 43.532, "profit_margins": 0.06341}
================================== Ai Message ==================================

{
  "stock": "6669.TW",
  "price_analysis": "近期股價呈現出波動上升趨勢，股價曾從2024年12月的1980逐步上升至2025年1月的2755，之後略有回調，目前在2050至2100之間波動。短期內，股價來回波動顯著，尤其是2025年1月的快速上升和隨後的下滑。",
  "technical_analysis": "技術指標目前顯示出一定的賣出壓力。RSI指標在33-45之間波動，顯示短期內可能存在超賣情況。隨機指標顯示出低於20的超賣狀態，尤其近日達到0。MACD指標與信號線間的負差距擴大，顯示出空方力量增強。此外，VWAP價格在現行股價之上，表明當前價格低於平均交易價位。",
  "financial_analysis": {
    "profitability_ratios": "淨利潤率為6.34%，表明公司的盈利能力略有受限，可能需要尋找提高效率及盈利的措施。",
    "liquidity_ratios": "流動性指標數據不可用，需進一步核查公司短期資產與負債的比重。",
    "solvency_ratios": "負債權益比為43.53%，表明償債能力尚可，可能需要注意負債增加對公司財務結構的影響。",
    "efficiency_ratios": "營運效率指標數據缺失，需進一步查詢存貨與應收賬款周轉狀況。",
    "market_ratios": "市盈率為14.48，市淨率為4.71，顯示市場對其未來增長的期望相對保守。",
    "summary": "總體而言，公司具備一定的盈利能力，但在運營效率及流